In [ ]:
import os
os.system("pip install sionna")
os.system("pip install tensorflow")

0

In [ ]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna

import numpy as np
import tensorflow as tf
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

# IPython "magic function" for inline plots
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
gpus = tf.config.list_physical_devices('GPU')
print('Number of GPUs available :', len(gpus))
if gpus:
    gpu_num = 0 # Index of the GPU to be used
    try:
        #tf.config.set_visible_devices([], 'GPU')
        tf.config.set_visible_devices(gpus[gpu_num], 'GPU')
        print('Only GPU number', gpu_num, 'used.')
        tf.config.experimental.set_memory_growth(gpus[gpu_num], True)
    except RuntimeError as e:
        print(e)

Number of GPUs available : 1
Only GPU number 0 used.


In [ ]:
!nvidia-smi

Tue Dec  3 08:34:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0              53W / 400W |    455MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
#get the zipped scenarios from the drive
drive.mount("/content/drive")

# !cp -r "/content/drive/My Drive/source_folder_path" "/content/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera
from sionna.channel import cir_to_ofdm_channel, subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel # CIRDataset

In [ ]:

"""Utility functions for Sionna"""
def remove_scene_UEs(scene):
    x_names = list(scene.receivers.keys())
    [scene.remove(x_names[i]) for i in range(len(x_names))]  # remove all RX
    # list comprehension to add users up to batch_size
    return scene


def add_scene_UEs(scene, N, positions, orientations):
    [scene.add(Receiver(name=f"rx00{i}", position=positions.reshape([N,3])[i, :], orientation=orientations.reshape([N,3])[i, :])) for i in range(N)]
    return scene

def scene2channels(scene, bandwidth, frequencies, max_depth=5, num_samples=10e5, apply_doppler=False,
                   channel_type="freq", en_diff=True, en_scat=False, scat_prob=1e-3, sampling_freq=50, num_time_steps=2,
                   L=16, K=16, rx_velocities=None):
    """
    This function generates the channels in a given scene. The number of channels in a scene is set to batch size provided by the config file. It generates channels between all TX and RX in the scene

    :param scene: scene with receivers and transmitters
    :param bandwidth:
    :param frequencies: OFDM subcarriers in baseband for the frequency channels
    :param max_depth: max number of reflections allowed
    :param num_samples: Number of rays to trace in order to generate candidates with the "fibonacci" method.
    :param apply_doppler: boolean to apply doppler or not
    :param channel_type: "time","freq", or "both"
    :param en_diff:
    :param en_scat:
    :param scat_prob: 1e-3 default
    :param sampling_freq:
    :param num_time_steps:
    :param L:
    :param K:
    :param rx_velocities:
    :return:
    """

    channel_type = channel_type.strip().lower()
    assert channel_type in ["time", "freq", "both"], print("channel_type must be one of ['time', 'freq', 'both']")

    # compute paths
    paths = scene.compute_paths(max_depth=max_depth, num_samples=num_samples, diffraction=en_diff, scattering=en_scat,
                                scat_keep_prob=scat_prob)

    if (apply_doppler):
        paths.apply_doppler(sampling_frequency=sampling_freq,  # 20ms samples
                            num_time_steps=num_time_steps,  # Number of OFDM symbols
                            rx_velocities=rx_velocities)  # rx speeds

    a, tau = paths.cir()


    # determine the channel taps
    l_min = -6
    l_max = L + l_min - 1

    if (channel_type == "time"):
        # l_min,l_max = sionna.channel.time_lag_discrete_time_channel(bandwidth, maximum_delay_spread=3e-06)
        h_time = sionna.channel.cir_to_time_channel(bandwidth, a, tau, l_min, l_max, normalize=False)
        return [h_time[0, :, :, 0, :, :, :], paths]  # strip to shape [U, Nr, Nt, T, L]
        # hm ([batch size, num_rx, num_rx_ant, num_tx, num_tx_ant, num_time_steps, l_max - l_min + 1], tf.complex) – Channel taps coefficients
    elif (channel_type == "freq"):

        # # # ##############################  cir_to_ofdm_channel(frequencies[:K], a, tau, normalize=False)  ####################################################
        # real_dtype = tau.dtype

        # if len(tau.shape) == 4:
        #     # Expand dims to broadcast with h. Add the following dimensions:
        #     #  - number of rx antennas (2)
        #     #  - number of tx antennas (4)
        #     tau = tf.expand_dims(tf.expand_dims(tau, axis=2), axis=4)
        #     # Broadcast is not supported yet by TF for such high rank tensors.
        #     # We therefore do part of it manually
        #     tau = tf.tile(tau, [1, 1, 1, 1, a.shape[4], 1])

        # # Add a time samples dimension for broadcasting
        # tau = tf.expand_dims(tau, axis=6)

        # # Bring all tensors to broadcastable shapes
        # tau = tf.expand_dims(tau, axis=-1)
        # h = tf.expand_dims(a, axis=-1)
        # frequencies = expand_to_rank(frequencies, tf.rank(tau), axis=0)

        # ## Compute the Fourier transforms of all cluster taps
        # # Exponential component
        # e = tf.exp(tf.complex(tf.constant(0, real_dtype),
        #     -2*PI*frequencies*tau))
        # h_f = tf.numpy_function(high_dim_mul, [h, e],tf.complex64)
        # # Sum over all clusters to get the channel frequency responses
        # h_freq = tf.reduce_sum(h_f, axis=-3)
        # # # ################################################################

        h_freq = cir_to_ofdm_channel(frequencies[:K], a, tau, normalize=False)
        # print(h_freq.shape)
        return [h_freq[0, :, :, 0, :, :, :], paths]  # strip to shape [U, Nr, Nt, T, K]
        # return [np.ones((2,16,64,4,4)),paths]
        # h_f ([batch size, num_rx, num_rx_ant, num_tx, num_tx_ant, num_time_steps, fft_size], tf.complex) – Channel frequency responses at frequencies
    elif(channel_type == "both"):
        h_freq = cir_to_ofdm_channel(frequencies[:K], a, tau, normalize=False)
        h_time = sionna.channel.cir_to_time_channel(bandwidth, a, tau, l_min, l_max, normalize=False)
        return [h_freq[0, :, :, 0, :, :, :], h_time[0, :, :, 0, :, :, :], paths]


def generate_channels(scene, UE_speed, antenna_loc, antenna_orient, bandwidth, frequencies, Nr=16, Nt=64, L=16, K=16,
                      batch_size=64, max_depth=5, num_samples=10e5, apply_doppler=False, channel_type='freq',
                      en_diff=True, en_scat=False, scat_prob=1e-5, sampling_freq=50, num_time_steps=2,
                      rx_velocities=None):
    """
    Generate channels for given UE locations, orientations and speeds

    :param scene: the scene to simulate radio environment
    :param UE_speed: (vx,vy,vz) speed of UE in the cartesian coordinates
    :param antenna_loc: (x,y,z) location of the antenna on the cartesian coordinates
    :param antenna_orient:
    :param bandwidth:
    :param frequencies:
    :param Nr:
    :param Nt:
    :param L:
    :param K:
    :param batch_size:
    :param max_depth:
    :param num_samples:
    :param apply_doppler:
    :param channel_type:
    :param en_diff:
    :param en_scat:
    :param scat_prob:
    :param sampling_freq:
    :param num_time_steps:
    :param rx_velocities:
    :return:
    """

    channel_type = channel_type.strip().lower()
    assert channel_type in ["time", "freq", "both"], print("channel_type must be one of ['time', 'freq', 'both']")

    dataset_size = antenna_loc.shape[0]

    # initialize the time and channel arrays
    channel_time = np.zeros((dataset_size, scene.rx_array.num_ant, scene.tx_array.num_ant, num_time_steps, L), dtype=np.complex64)
    channel_freq = np.zeros((dataset_size, scene.rx_array.num_ant, scene.tx_array.num_ant, num_time_steps, K), dtype=np.complex64)
    paths = []

    # remove old UEs
    scene = remove_scene_UEs(scene)

    x_names = list(scene.receivers.keys())
    loops = int(np.ceil(dataset_size / batch_size))
    batch_per_loop = batch_size * np.ones((loops))
    batch_per_loop[-1] = dataset_size - np.sum(batch_per_loop[:-1])  # last may not be a full batch
    ds_counter = 0
    for loop in range(loops):
        active_b = int(batch_per_loop[loop])

        # select UEs and their corresponding features for one batch
        antenna_loc_batch = antenna_loc[ds_counter:ds_counter + active_b, :].squeeze()
        antenna_orient_batch = antenna_orient[ds_counter:ds_counter + active_b, :].squeeze()
        UE_speed_batch = np.expand_dims(UE_speed[ds_counter:ds_counter + active_b, :],
                                        axis=1)  # for the rx_velocities parameter in apply_doppler

        # add the UEs to the scene
        scene = add_scene_UEs(scene, active_b, antenna_loc_batch, antenna_orient_batch)

        print(f"Generating channels for UEs {ds_counter}-{ds_counter + active_b} out of {dataset_size}")
        channels = scene2channels(scene, bandwidth, frequencies, max_depth, num_samples, apply_doppler, channel_type,
                                  en_diff, en_scat, scat_prob, sampling_freq, num_time_steps, L, K, UE_speed_batch)

        if (channel_type == "freq"):
            channel_freq[ds_counter:ds_counter + active_b] = channels[0].numpy()
            # add paths to the paths
            paths.append(channels[1])
        elif (channel_type == "time"):
            channel_time[ds_counter:ds_counter + active_b] = channels[0].numpy()
            # add paths to the paths
            paths.append(channels[1])
        else:
            channel_freq[ds_counter:ds_counter + active_b] = channels[0].numpy()
            channel_time[ds_counter:ds_counter + active_b] = channels[1].numpy()
            # add paths to the paths
            paths.append(channels[2])

        # increase the counter
        ds_counter += active_b

        # remove old UEs
        scene = remove_scene_UEs(scene)

    if (channel_type == "freq"):
        return [channel_freq, paths]
    elif (channel_type == "time"):
        return [channel_time, paths]
    else:
        return [channel_freq, channel_time, paths]


def save_parameters(file_path, **kwargs):
    # Initialize an empty list to store parameters
    # rt_data = []

    # # Append each parameter to the list
    # for key, value in kwargs.items():
    #     rt_data.append((key, value))

    rt_data = {}

    # Append each parameter to the list
    for key, value in kwargs.items():
        rt_data[key] = value

    # Save the list using pickle
    with open(file_path, 'wb') as f:
        pickle.dump(rt_data, f)

    print(f"Saved to {file_path}")


def load_parameters(file_path):
    target_dict = {}
    # Load the list from the pickle file
    with open(file_path, 'rb') as f:
        parameters_list = pickle.load(f)

    # Assign each parameter to the target dictionary
    for key, value in parameters_list:
        target_dict[key] = value
    return target_dict

def save_simulation_parameters(yaml_file,pdf_file):
    # Read the YAML file
    with open(yaml_file, 'r') as file:
        data = yaml.safe_load(file)

    # Create a PDF file
    c = canvas.Canvas(pdf_file, pagesize=letter)
    width, height = letter

    # Set up starting position for text
    y_position = height - 40
    line_height = 15  # Space between lines
    margin = 30  # Left margin

    # Function to draw text with indentation and dashes
    def draw_text(text, indent_level=0):
        nonlocal y_position

        # Check if we need to start a new page
        if y_position < 40:  # Leave space for footer
            c.showPage()
            c.setFont("Helvetica", 10)  # Reset font for new page
            y_position = height - 40  # Reset y position for new page

        # Create a dynamic length of dashes based on indent level
        dashes = '-' * (indent_level + 3)  # Length increases with indentation
        indent = "    " * indent_level  # Indentation for hierarchical structure

        # Draw the text with dashes at the beginning
        c.drawString(margin + indent_level * 10, y_position, f"{dashes} {indent}{text}")
        y_position -= line_height  # Move down for the next line

    # Recursively format and draw the YAML content
    def format_yaml(data, indent_level=0):
        if isinstance(data, dict):
            for key, value in data.items():
                # Format key-value pairs
                draw_text(f"{key}: {value if not isinstance(value, (list, dict)) else ''}", indent_level)
                if isinstance(value, (dict, list)):  # Only call recursively for dicts and lists
                    format_yaml(value, indent_level + 1)  # Increase indentation for nested items
        elif isinstance(data, list):
            for item in data:
                draw_text(f"- {item}", indent_level)  # Bullet point for list items
        else:
            draw_text(f"{data}", indent_level)  # Just print the value

    c.setFont("Helvetica", 10)  # Set font for the PDF
    format_yaml(data)

    # Save the PDF
    c.save()


"""Utility functions for SUMO and Sionna data integration"""
def antenna_orientation(sumo_angles, type):
    """

    :param sumo_angles: angles in degrees
    :param type: type of orientation
    :return: [rx,ry,rz] in angles
    """

    assert type.strip().lower() in ["vertical", "horizontal"], print("Antenna orientation type not supported. Select either 'vertical' or 'horizontal'")
    orientation = np.zeros((sumo_angles.shape[0], 3))
    if type.strip().lower() == "vertical":
        # antenna normal is parallel to the ground looking rightwards (+x)
        orientation[:, 2] = -1 * sumo_angles
    elif type.strip().lower() == "horizontal":
        # antenna normal is perpendicular to the ground, antenna looking towards sky
        orientation[:, 1] = -90
        orientation[:, 2] = -1 * sumo_angles
    return orientation


def antenna_position_offset(sumo_angles, antenna_offset):
    """

    :param sumo_angles: the angle of the vehicle based on the global coordinate system in SUMO and Sionna. When the vehicle is facing towards +y and positioned in xy-plane then the angle is 0. The angle is clockwise starting from +y direction.
    :param antenna_offset: position offset of the antenna, when vehicle is facing +y direction and centered at x = 0, and bottom of the vehicle is z = 0. The offset is with respect to the center of the front bumper of the car.
    :return:
    """
    dx, dy, dz = antenna_offset[0], antenna_offset[1], antenna_offset[2]

    dy_hat = np.expand_dims(np.cos(sumo_angles / 180 * np.pi) * dy, axis=1)
    dx_hat = np.expand_dims(np.sin(sumo_angles / 180 * np.pi) * dy, axis=1)
    dz_hat = dz * np.ones((sumo_angles.shape[0], 1))

    return np.hstack([dx_hat, dy_hat, dz_hat])


def get_vehicle_speed(sumo_angles, vehicle_speed):
    """
    This function is to calculate the speed values with respect to Cartesian coordinates of SUMO and Sionna
    :param sumo_angles: Angles described in antenna_position_offset function
    :param vehicle_speed: Vehicle speed based on the vehicle's pointing direction
    :return:
    """

    # rx_velocities : [batch_size, num_tx, 3] or broadcastable, tf.float: num_tx is 1 in our case
    vx, vy, vz = np.expand_dims(np.multiply(vehicle_speed, np.sin(sumo_angles / 180 * np.pi)), axis=1), np.expand_dims(
        np.multiply(vehicle_speed, np.cos(sumo_angles / 180 * np.pi)), axis=1), np.zeros((sumo_angles.shape[0], 1))
    return np.hstack([vx, vy, vz])


def get_UE_information(sumo_data, antenna_orient, antenna_offset):
    """
    This function returns location and antenna orientation of each UE that communicate with base station
    :param sumo_data: dataframe of the current timestep
    :return: np.array of UE location, UE_speed, antenna antenna orientations based on Sionna,
    """
    UE_data = sumo_data.loc[sumo_data.vehicle_id.str.contains("car", case=False), :]
    UE_loc = UE_data.loc[:, ["x", "y", "z"]].values
    UE_angle = UE_data.angle.values

    # get the antenna orientation based on the vehicle orientation
    antenna_orient = antenna_orientation(UE_angle, antenna_orient)
    # convert the antenna orientation [rx,ry,rz] in angles to [alpha,beta,gamma] in radians
    antenna_orient = antenna_orient[:, ::-1] * np.pi / 180.0

    # get the antenna location based on the vehicle orientation
    antenna_loc = UE_loc + antenna_position_offset(UE_angle, antenna_offset)

    # get the vehicle speed for each axis based on the vehicle speed and the orientation
    UE_speed = get_vehicle_speed(UE_angle, UE_data.speed.values)

    # get the vehicle ID
    UE_vehicle_id = UE_data.vehicle_id.values
    return UE_loc, UE_speed, antenna_loc, antenna_orient, UE_vehicle_id

In [ ]:
scenarios_channel_path = f"/content/drive/MyDrive/ECE257A_Final_project/Dataset"

if(not os.path.isdir(scenarios_channel_path)):
  os.makedirs(scenarios_channel_path)

In [ ]:
lanes = [[[0,402.73],[135.97,138.33]],[[247.7,246.41],[32.71,225.91]]]
steps = [['x',0.2],['y',0.2]]
fcs = [2.5e9,2.62e9]
antenna_configs = [[8,8],[8,4]]
subcarss = [80,160]

In [ ]:
scene_dir = "drive/MyDrive/AsilomarDataset/necessary_files_new/base_scene/base_scene.xml"
scene = load_scene(scene_dir)

In [ ]:
my_cam = Camera("my_cam", position=[234,83,1500], look_at=[234,83,0])
scene.remove("my_cam")
scene.add(my_cam)

# Render scene with new camera*
scene.render_to_file(camera="my_cam",resolution = [800, 400],num_samples=512,filename=f"{scenarios_channel_path}//scene.png") # Increase num_samples to increase image quality

In [ ]:
batch_size = 64

In [ ]:
Nrx = 1
Nry = 1
save_paths = False
lanes = 40
for fc in fcs:
  for antenna in antenna_configs:
    Ntx,Nty = antenna[0],antenna[1]
    for no_of_subcarriers in subcarss:
      subcar_spacing = bandwidth/no_of_subcarriers
      frequencies = subcarrier_frequencies(no_of_subcarriers, subcar_spacing) # multiply by 12 for resource blocks
      for lane_id in range(lanes):

        if([fc,Ntx,Nty,no_of_subcarriers,lane_id] not in generated_ones):

          batch_size = 64/int(no_of_subcarriers/80*Ntx*Nty/32)
          print(f"Batch size = {batch_size}")
          #add the receiver adn transmitter to the scene
          scene.remove("tx")
          scene.remove("rx")

          x_range,y_range = lanes[lane_id]
          print(f"Fc = {fc}, Lane {lanes[lane_id]}, K = {no_of_subcarriers}, BW = {bandwidth}, Ntx,Nty = {Ntx},{Nty}")
          if(steps[lane_id][0] == 'x'):
            x_pos = np.arange(x_range[0],x_range[1],steps[lane_id][1])
            y_pos = np.linspace(y_range[0],y_range[1],len(x_pos))
          else:
            y_pos = np.arange(y_range[0],y_range[1],steps[lane_id][1])
            x_pos = np.linspace(x_range[0],x_range[1],len(y_pos))

          z_pos = np.ones_like(x_pos)*1.5
          antenna_pos = np.hstack([x_pos.reshape(-1,1),y_pos.reshape(-1,1),z_pos.reshape(-1,1)])
          antenna_orient = np.zeros_like(antenna_pos)
          UE_speed = np.zeros_like(antenna_pos)
          UE_speed[:,0] = np.random.uniform(9.5,10,size=len(x_pos))

          #set the carrier frequency of the simulation
          scene.frequency = fc

          #configure the scene for the channel generation
          tx = Transmitter(name="tx", position=Tx_pos, orientation=Tx_orient)
          rx = Receiver(name="rx", position = [2,46, 1.5])

          #add the receiver adn transmitter to the scene
          scene.add(tx)
          scene.add(rx)

          scene.synthetic_array = True
          # update the arrays
          scene.tx_array = PlanarArray(num_rows=Nty,
                                      num_cols=Ntx,
                                      vertical_spacing=Tdy,
                                      horizontal_spacing=Tdx,
                                      pattern="tr38901",
                                      polarization="V")

          # Configure antenna array for all receivers
          scene.rx_array = PlanarArray(num_rows=Nry,
                                      num_cols=Nrx,
                                      vertical_spacing=Tdy,
                                      horizontal_spacing=Tdx,
                                      pattern="tr38901",
                                      polarization="V")

          #generate channels based on the antenna locations, orientations and vehicle speed
          rt_data = generate_channels(scene,
                                        UE_speed,
                                        antenna_pos,
                                        antenna_orient,
                                        bandwidth,
                                        frequencies,
                                        Nr=Nrx*Nry,
                                        Nt=Ntx*Nty,
                                        L=L,
                                        K=no_of_subcarriers,
                                        batch_size=batch_size,
                                        max_depth=max_depth,
                                        num_samples=num_samples,
                                        apply_doppler=apply_doppler,
                                        channel_type=channel_type,
                                        en_diff=en_diff,
                                        en_scat=en_scat,
                                        scat_prob=scat_prob,
                                        sampling_freq=sampling_freq,
                                        num_time_steps=num_time_steps,
                                        rx_velocities=UE_speed)

          #store the channels for the scenarios
          file_path = os.path.join(scenarios_channel_path,f'{lane_id+1}Lane_{antenna_pos.shape[0]}U_{Nrx*Nry}Rx_{Ntx*Nty}Tx_{num_time_steps}T_{no_of_subcarriers}K_{fc}fc.pickle')


          generated_ones.append([fc,Ntx,Nty,no_of_subcarriers,lane_id])
          if(channel_type =="time"):
              if(save_paths == True):
                  save_parameters(file_path,time_channel=rt_data[0],paths=rt_data[1],UE_loc=UE_loc,UE_speed=UE_speed,antenna_loc=antenna_loc,antenna_orient=antenna_orient,UE_vehicle_id=UE_id)
              else:
                  save_parameters(file_path,time_channel=rt_data[0],UE_loc=UE_loc,UE_speed=UE_speed,antenna_loc=antenna_loc,antenna_orient=antenna_orient,UE_vehicle_id=UE_id)
          elif(channel_type=="freq"):
              if(save_paths == True):
                  save_parameters(file_path,freq_channel=rt_data[0],paths=rt_data[1],UE_loc=UE_loc,UE_speed=UE_speed,antenna_loc=antenna_loc,antenna_orient=antenna_orient,UE_vehicle_id=UE_id)
              else:
                  save_parameters(file_path,freq_channel=rt_data[0],UE_loc=antenna_pos,UE_speed=UE_speed,antenna_orient=antenna_orient)
          else:
              if(save_paths == True):
                  save_parameters(file_path,freq_channel=rt_data[0],time_channel =rt_data[1],paths=rt_data[2],UE_loc=UE_loc,UE_speed=UE_speed,antenna_loc=antenna_loc,antenna_orient=antenna_orient,UE_vehicle_id=UE_id)
              else:
                  save_parameters(file_path,freq_channel=rt_data[0],time_channel=rt_data[1],UE_loc=UE_loc,UE_speed=UE_speed,antenna_loc=antenna_loc,antenna_orient=antenna_orient,UE_vehicle_id=UE_id)

Batch size = 16.0
Fc = 2500000000.0, Lane [[0, 402.73], [135.97, 138.33]], K = 160, BW = 8000000.0, Ntx,Nty = 8,8
Generating channels for UEs 0-16 out of 2014
Generating channels for UEs 16-32 out of 2014
Generating channels for UEs 32-48 out of 2014
Generating channels for UEs 48-64 out of 2014
Generating channels for UEs 64-80 out of 2014
Generating channels for UEs 80-96 out of 2014
Generating channels for UEs 96-112 out of 2014
Generating channels for UEs 112-128 out of 2014
Generating channels for UEs 128-144 out of 2014
Generating channels for UEs 144-160 out of 2014
Generating channels for UEs 160-176 out of 2014
Generating channels for UEs 176-192 out of 2014
Generating channels for UEs 192-208 out of 2014
Generating channels for UEs 208-224 out of 2014
Generating channels for UEs 224-240 out of 2014
Generating channels for UEs 240-256 out of 2014
Generating channels for UEs 256-272 out of 2014
Generating channels for UEs 272-288 out of 2014
Generating channels for UEs 288-304 

ResourceExhaustedError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul] name: 